In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text('src_base_dir', '', label='Enter Source Base Dir')
dbutils.widgets.text('bronze_base_dir', '', label='Enter Target Base Dir')
dbutils.widgets.text('ds', '', label='Enter Dataset Name')

In [0]:
src_base_dir = dbutils.widgets.get('src_base_dir')

In [0]:
bronze_base_dir = dbutils.widgets.get('bronze_base_dir')

In [0]:
ds = dbutils.widgets.get('ds')

In [0]:
import json

def get_columns(schemas_file, ds_name):
    schema_text = spark.read.text(schemas_file, wholetext=True).first().value
    schemas = json.loads(schema_text)
    column_details = schemas[ds_name]
    columns = [col['column_name'] for col in sorted(column_details, key=lambda col: col['column_position'])]
    return columns

In [0]:
ds

Out[17]: 'orders'

In [0]:
print(f'Processing {ds} data')
columns = get_columns(f'dbfs:{src_base_dir}/schemas.json', ds)
df = spark. \
    read. \
    csv(f'{src_base_dir}/{ds}', inferSchema=True). \
    toDF(*columns)

Processing orders data


In [0]:
df.show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
|       6|2013-07-25 00:00:00|             7130|       COMPLETE|
|       7|2013-07-25 00:00:00|             4530|       COMPLETE|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:0

In [0]:
df.write. \
    mode('overwrite'). \
    parquet(f'{bronze_base_dir}/{ds}')